In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
import os
os.chdir('./drive/My Drive/TESI/STAR')

Mounted at /content/drive/


In [2]:
from __future__ import print_function
import os
import _pickle as pickle
import numpy as np
import math
import h5py

from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint,TensorBoard, LearningRateScheduler
from star.model import *
import star.metrics as metrics
from star import TaxiNYC

np.random.seed(1337)  # for reproducibility

In [3]:
# parameters
DATAPATH = '../data' 
nb_epoch = 100  # number of epoch at training stage
# nb_epoch_cont = 150  # number of epoch at training (cont) stage
batch_size = 16  # batch size
T = 24  # number of time intervals in one day
CACHEDATA = True  # cache data or NOT

lr = 0.00015  # learning rate
len_closeness = 3  # length of closeness dependent sequence
len_period = 1  # length of peroid dependent sequence
len_trend = 1  # length of trend dependent sequence
nb_residual_unit = 2   # number of residual units

nb_flow = 2  # there are two types of flows: new-flow and end-flow
# divide data into two subsets: Train & Test, 
days_test = 7*4
len_test = T*days_test
len_val = 2*len_test

map_height, map_width = 16, 8  # grid size

path_cache = os.path.join(DATAPATH, 'CACHE', 'STAR')  # cache path
path_result = 'RET'
path_model = 'MODEL'
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [4]:
def build_model(external_dim, save_model_pic=False):
    c_conf = (len_closeness, nb_flow, map_height,
              map_width) if len_closeness > 0 else None
    p_conf = (len_period, nb_flow, map_height,
              map_width) if len_period > 0 else None
    t_conf = (len_trend, nb_flow, map_height,
              map_width) if len_trend > 0 else None

    model = STAR(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                     external_dim=external_dim, nb_residual_unit=nb_residual_unit)
    opt = Adam(lr=lr)
    # opt = SGD(lr=lr)
    model.compile(loss='mse', optimizer=opt, metrics=[metrics.rmse])
    # model.summary()
    if (save_model_pic):
        from keras.utils.vis_utils import plot_model
        plot_model(model, to_file='TaxiNYC_model.png', show_shapes=True)

    return model

In [5]:
def read_cache(fname):
    mmn = pickle.load(open('preprocessing_taxinyc.pkl', 'rb'))

    f = h5py.File(fname, 'r')
    num = int(f['num'].value)
    X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test = [], [], [], [], [], [], [], []
    for i in range(num):
        X_train_all.append(f['X_train_all_%i' % i].value)
        X_train.append(f['X_train_%i' % i].value)
        X_val.append(f['X_val_%i' % i].value)
        X_test.append(f['X_test_%i' % i].value)
    Y_train_all = f['Y_train_all'].value
    Y_train = f['Y_train'].value
    Y_val = f['Y_val'].value
    Y_test = f['Y_test'].value
    external_dim = f['external_dim'].value
    timestamp_train_all = f['T_train_all'].value
    timestamp_train = f['T_train'].value
    timestamp_val = f['T_val'].value
    timestamp_test = f['T_test'].value
    f.close()

    return X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test, mmn, external_dim, timestamp_train_all, timestamp_train, timestamp_val, timestamp_test

def cache(fname, X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test, external_dim, timestamp_train_all, timestamp_train, timestamp_val, timestamp_test):
    h5 = h5py.File(fname, 'w')
    h5.create_dataset('num', data=len(X_train_all))

    for i, data in enumerate(X_train_all):
        h5.create_dataset('X_train_all_%i' % i, data=data)
    for i, data in enumerate(X_train):
        h5.create_dataset('X_train_%i' % i, data=data)
    for i, data in enumerate(X_val):
        h5.create_dataset('X_val_%i' % i, data=data)
    # for i, data in enumerate(Y_train):
    for i, data in enumerate(X_test):
        h5.create_dataset('X_test_%i' % i, data=data)

    h5.create_dataset('Y_train_all', data=Y_train_all)
    h5.create_dataset('Y_train', data=Y_train)
    h5.create_dataset('Y_val', data=Y_val)
    h5.create_dataset('Y_test', data=Y_test)
    external_dim = -1 if external_dim is None else int(external_dim)
    h5.create_dataset('external_dim', data=external_dim)
    h5.create_dataset('T_train_all', data=timestamp_train_all)
    h5.create_dataset('T_train', data=timestamp_train)
    h5.create_dataset('T_val', data=timestamp_val)
    h5.create_dataset('T_test', data=timestamp_test)
    h5.close()


In [6]:
tf.keras.backend.set_image_data_format('channels_first')
tf.keras.backend.image_data_format()

'channels_first'

In [7]:
print("loading data...")
fname = os.path.join(path_cache, 'TaxiNYC_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = read_cache(
        fname)
    print("load %s successfully" % fname)
else:
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = TaxiNYC.load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        len_val=len_val, preprocess_name='preprocessing_taxinyc.pkl', meta_data=True, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test,
              external_dim, timestamp_train_all, timestamp_train, timestamp_val, timestamp_test)

print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])

print('=' * 10)

loading data...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if __name__ == '__main__':


load ../data/CACHE/STAR/TaxiNYC_C3_P1_T1.h5 successfully

 days (test):  [b'20141204', b'20141205', b'20141206', b'20141207', b'20141208', b'20141209', b'20141210', b'20141211', b'20141212', b'20141213', b'20141214', b'20141215', b'20141216', b'20141217', b'20141218', b'20141219', b'20141220', b'20141221', b'20141222', b'20141223', b'20141224', b'20141225', b'20141226', b'20141227', b'20141228', b'20141229', b'20141230', b'20141231']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: H5pyDeprecationWarning: dataset.v

In [8]:
X_val[0].mean()

-0.9795904128831029

In [9]:
def subset(train_set, num_timestamps):
    if isinstance(train_set,list):
        new_train_set = []
        new_train_set.append(train_set[0][-num_timestamps:])
        new_train_set.append(train_set[1][-num_timestamps:])
    else:
        new_train_set = train_set[-num_timestamps:]
    return new_train_set

dic_rmse = {}
for i in range(0,1):
    
    print("compiling model...")

    # lr_callback = LearningRateScheduler(lrschedule)

    model = build_model(external_dim, save_model_pic=False)

    hyperparams_name = 'TaxiNYC.c{}.p{}.t{}.resunit{}.iter{}'.format(
        len_closeness, len_period, len_trend, nb_residual_unit, i)
    fname_param = os.path.join(path_model, '{}.best.h5'.format(hyperparams_name))
    print(hyperparams_name)

    early_stopping = EarlyStopping(monitor='val_rmse', patience=24, mode='min')
    model_checkpoint = ModelCheckpoint(
        fname_param, monitor='val_rmse', verbose=1, save_best_only=True, mode='min')

    print('=' * 10)
    print("training model...")
    history = model.fit(#subset(X_train, 4000), subset(Y_train, 4000),
                        X_train, Y_train,
                        epochs=nb_epoch,
                        batch_size=batch_size,
                        validation_data=(X_val,Y_val),
                        callbacks=[early_stopping, model_checkpoint],
                        verbose=1)
    model.save_weights(os.path.join(
        path_model, '{}.h5'.format(hyperparams_name)), overwrite=True)
    pickle.dump((history.history), open(os.path.join(
        path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))

    print('=' * 10)
    print('evaluating using the model that has the best loss on the valid set')

    model.load_weights(fname_param)
    
    score = model.evaluate(
        X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
    print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
          (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2. ))

compiling model...
TaxiNYC.c3.p1.t1.resunit2.iter0
training model...
Epoch 1/100
2469/2469 [==============================] - ETA: 0s - loss: 0.0113 - rmse: 0.0423
Epoch 00001: val_rmse improved from inf to 0.01208, saving model to MODEL/TaxiNYC.c3.p1.t1.resunit2.iter0.best.h5
2469/2469 [==============================] - 11s 4ms/step - loss: 0.0113 - rmse: 0.0423 - val_loss: 1.4898e-04 - val_rmse: 0.0121
Epoch 2/100
2463/2469 [============================>.] - ETA: 0s - loss: 1.2281e-04 - rmse: 0.0108
Epoch 00002: val_rmse improved from 0.01208 to 0.00958, saving model to MODEL/TaxiNYC.c3.p1.t1.resunit2.iter0.best.h5
2469/2469 [==============================] - 10s 4ms/step - loss: 1.2269e-04 - rmse: 0.0107 - val_loss: 9.3961e-05 - val_rmse: 0.0096
Epoch 3/100
2467/2469 [============================>.] - ETA: 0s - loss: 8.7664e-05 - rmse: 0.0090
Epoch 00003: val_rmse improved from 0.00958 to 0.00839, saving model to MODEL/TaxiNYC.c3.p1.t1.resunit2.iter0.best.h5
2469/2469 [=============